In [1]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [2]:
results = requests.get("https://api.nytimes.com/svc/books/v3/lists.json?list=mass-market-monthly&api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb")

In [3]:
results.status_code

200

In [4]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "mass-market-monthly"
API_KEY = "keys.json"

def subtract_months(date, months):
    year = date.year
    month = date.month
    new_month = month - months
    if new_month <= 0:
        year -= (abs(new_month) // 12) + 1
        new_month = 12 - (abs(new_month) % 12)
    return date.replace(year=year, month=new_month)

bestsellers = []
months = []
num_calls_to_make = 1
months_per_call = 43  # Number of months to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * months_per_call
    start_date = datetime.date(2023, 4, 23)
    start_date = subtract_months(start_date, ctr)
    end_date = subtract_months(start_date, months_per_call)
    
    for i in range(months_per_call):
        curr_date = subtract_months(start_date, i)
        print(curr_date)
        url = f"{BOOKS_ROOT}/{curr_date}/{LIST}.json?api-key={API_KEY}"
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            for book in results['results']['books']:
                book_info = {
                    'title': book['title'],
                    'description': book['description'],
                    'contributor': book['contributor'],
                    'author': book['author'],
                    'contributor_note': book['contributor_note'],
                    'price': book['price'],
                    'age_group': book['age_group'],
                    'publisher': book['publisher'],
                    'primary_isbn13': book['primary_isbn13'],
                    'primary_isbn10': book['primary_isbn10'],
                    'rank': book['rank'],
                    'rank_last_week': book['rank_last_week'],
                    'weeks_on_list': book['weeks_on_list'],
                    'asterisk': book['asterisk'],
                    'dagger': book['dagger']
                }
                bestsellers.append(book_info)
            months.append(str(curr_date))
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)

2023-04-23
https://api.nytimes.com/svc/books/v3/lists/2023-04-23/mass-market-monthly.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-03-23
https://api.nytimes.com/svc/books/v3/lists/2023-03-23/mass-market-monthly.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-02-23
https://api.nytimes.com/svc/books/v3/lists/2023-02-23/mass-market-monthly.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-01-23
https://api.nytimes.com/svc/books/v3/lists/2023-01-23/mass-market-monthly.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2022-12-23
https://api.nytimes.com/svc/books/v3/lists/2022-12-23/mass-market-monthly.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2022-11-23
https://api.nytimes.com/svc/books/v3/lists/2022-11-23/mass-market-monthly.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2022-10-23
https://api.nytimes.com/svc/books/v3/lists/2022-10-23/mass-market-monthly.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2022-09-23
https://api.nytimes.com/svc/books/v3/lists/2022-09-23/mass-market-monthl

In [23]:
mass_market_monthly = pd.json_normalize(bestsellers)

In [24]:
mass_market_monthly.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,DEATH OF THE BLACK WIDOW,Detective Walter O’Brien is obsessed with an e...,by James Patterson and J.D. Barker,James Patterson and J.D. Barker,,0.00,,Grand Central,9781538753088,1538753081,1,0,0,0,0
1,MERCY,The fourth book in the Atlee Pine series. Atle...,by David Baldacci,David Baldacci,,0.00,,Grand Central,9781538719718,1538719711,2,0,0,0,0
2,TOM CLANCY: ZERO HOUR,Jack Ryan Jr. appears to be the only person wh...,by Don Bentley,Don Bentley,,0.00,,Berkley,9780593422748,0593422740,3,0,0,0,0
3,GETTING EVEN,Two suspense stories: “Yesterday’s Lies” and “...,by Lisa Jackson,Lisa Jackson,,0.00,,Zebra,9781420155549,1420155547,4,0,0,0,0
4,ONE NIGHT,A tale of two rival workaholic radio broadcast...,by Debbie Macomber,Debbie Macomber,,0.00,,Avon,9780061081859,006108185X,5,0,0,0,0


In [25]:
mass_market_monthly.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
640,COTTAGE BY THE SEA,Annie Marlow forms new relationships in the Pa...,by Debbie Macomber,Debbie Macomber,,0.00,,Ballantine,9780399181276,,11,0,0,0,0
641,A COLUMN OF FIRE,A pair of lovers find themselves on opposite s...,by Ken Follett,Ken Follett,,0.00,,Penguin,9780451478009,0451478002,12,0,0,0,0
642,THE ART OF RACING IN THE RAIN,"An insightful Lab-terrier mix helps his owner,...",by Garth Stein,Garth Stein,,0.00,,Harper,9780062364913,006236491X,13,0,0,0,0
643,THE COUPLE NEXT DOOR,A couple’s secrets emerge after their baby dis...,by Shari Lapena,Shari Lapena,,0.00,,Penguin,9780525505310,0525505318,14,0,0,0,0
644,THE STALKING,The 29th book in the Krewe of Hunters series. ...,by Heather Graham,Heather Graham,,0.00,,MIRA,9780778308119,0778308111,15,0,0,0,0


In [26]:
len(mass_market_monthly)

645

In [12]:
len(months)

43

In [16]:
#repeat months 15 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 15
repeated_months = []
for element in months:
    for i in range(k):
       repeated_months.append(element)

In [17]:
len(repeated_months)

645

In [27]:
mass_market_monthly['month'] = repeated_months

In [28]:
mass_market_monthly.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,month
0,DEATH OF THE BLACK WIDOW,Detective Walter O’Brien is obsessed with an e...,by James Patterson and J.D. Barker,James Patterson and J.D. Barker,,0.00,,Grand Central,9781538753088,1538753081,1,0,0,0,0,2023-04-23
1,MERCY,The fourth book in the Atlee Pine series. Atle...,by David Baldacci,David Baldacci,,0.00,,Grand Central,9781538719718,1538719711,2,0,0,0,0,2023-04-23
2,TOM CLANCY: ZERO HOUR,Jack Ryan Jr. appears to be the only person wh...,by Don Bentley,Don Bentley,,0.00,,Berkley,9780593422748,0593422740,3,0,0,0,0,2023-04-23
3,GETTING EVEN,Two suspense stories: “Yesterday’s Lies” and “...,by Lisa Jackson,Lisa Jackson,,0.00,,Zebra,9781420155549,1420155547,4,0,0,0,0,2023-04-23
4,ONE NIGHT,A tale of two rival workaholic radio broadcast...,by Debbie Macomber,Debbie Macomber,,0.00,,Avon,9780061081859,006108185X,5,0,0,0,0,2023-04-23


In [29]:
mass_market_monthly.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,month
640,COTTAGE BY THE SEA,Annie Marlow forms new relationships in the Pa...,by Debbie Macomber,Debbie Macomber,,0.00,,Ballantine,9780399181276,,11,0,0,0,0,2019-10-23
641,A COLUMN OF FIRE,A pair of lovers find themselves on opposite s...,by Ken Follett,Ken Follett,,0.00,,Penguin,9780451478009,0451478002,12,0,0,0,0,2019-10-23
642,THE ART OF RACING IN THE RAIN,"An insightful Lab-terrier mix helps his owner,...",by Garth Stein,Garth Stein,,0.00,,Harper,9780062364913,006236491X,13,0,0,0,0,2019-10-23
643,THE COUPLE NEXT DOOR,A couple’s secrets emerge after their baby dis...,by Shari Lapena,Shari Lapena,,0.00,,Penguin,9780525505310,0525505318,14,0,0,0,0,2019-10-23
644,THE STALKING,The 29th book in the Krewe of Hunters series. ...,by Heather Graham,Heather Graham,,0.00,,MIRA,9780778308119,0778308111,15,0,0,0,0,2019-10-23


In [30]:
#and a column indicating which list these come from
k2 = 645
monthly_fiction_list = ['Mass Market Monthly']
nyt_list_monthly_fiction = []

for element in monthly_fiction_list:
    for i in range(k2):
        nyt_list_monthly_fiction.append(element)

len(nyt_list_monthly_fiction)

645

In [31]:
mass_market_monthly['nyt_list'] = nyt_list_monthly_fiction

mass_market_monthly.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,month,nyt_list
625,PLAY DIRTY,A fallen football star agrees to secretly fath...,by Sandra Brown,Sandra Brown,,0.00,,Pocket,9781982121112,1982121114,11,0,0,0,0,2019-11-23,Mass Market Monthly
626,VINCE FLYNN: RED WAR,When the Russian prime minister plots to invad...,by Kyle Mills,Kyle Mills,,0.00,,Pocket,9781501190605,1501190601,12,3,0,0,0,2019-11-23,Mass Market Monthly
627,SEASON OF WONDER,The ninth book in the Haven Point series. Dani...,by RaeAnne Thayne,RaeAnne Thayne,,0.00,,HQN,9781335041470,1335041478,13,0,0,0,0,2019-11-23,Mass Market Monthly
628,IMMORTAL BORN,The 30th book in the Argeneau series. Magnus m...,by Lynsay Sands,Lynsay Sands,,0.00,,Avon,9780062855213,0062855212,14,0,0,0,0,2019-11-23,Mass Market Monthly
629,BULLETS DON'T ARGUE,The third book in the Perley Gates Western ser...,by William W. Johnstone and J.A. Johnstone,William W Johnstone and JA Johnstone,,0.00,,Pinnacle,9780786043668,0786043660,15,0,0,0,0,2019-11-23,Mass Market Monthly
630,LOOK ALIVE TWENTY-FIVE,The 25th book in the Stephanie Plum series. Wh...,by Janet Evanovich,Janet Evanovich,,0.00,,Putnam,9780399179242,0399179240,1,0,0,0,0,2019-10-23,Mass Market Monthly
631,THE RECKONING,A decorated World War II veteran shoots and ki...,by John Grisham,John Grisham,,0.00,,Dell,9780525620938,0525620931,2,0,0,0,0,2019-10-23,Mass Market Monthly
632,VINCE FLYNN: RED WAR,When the Russian prime minister plots to invad...,by Kyle Mills,Kyle Mills,,0.00,,Pocket,9781501190605,1501190601,3,0,0,0,0,2019-10-23,Mass Market Monthly
633,SHADOW TYRANTS,Juan Cabrillo and his crew fight two destructi...,by Clive Cussler and Boyd Morrison,Clive Cussler and Boyd Morrison,,0.00,,Putnam,9780735219083,0735219087,4,0,0,0,0,2019-10-23,Mass Market Monthly
634,JUROR #3,Ruby Bozarth defends a college football star c...,by James Patterson and Nancy Allen,James Patterson and Nancy Allen,,0.00,,Vision,9781538713792,1538713799,5,0,0,0,0,2019-10-23,Mass Market Monthly


In [33]:
mass_market_monthly.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2019to2023_mass_market_monthly.csv", index=False)